## Deliverable 2. Create a Customer Travel Destinations Map.

In [19]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import sys
sys.path.append('../')
from config import g_key

In [20]:
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [22]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Cidreira,-30.1811,-50.2056,66.85,74,9,8.61,BR,2022-02-07 04:49:48,Clear
1,1,Sisimiut,66.9395,-53.6735,3.72,94,100,20.04,GL,2022-02-07 04:46:14,Snow
2,2,Soddy-Daisy,35.2359,-85.1908,33.53,87,0,3.20,US,2022-02-07 04:49:49,Clear
3,3,Bam,29.1060,58.3570,52.72,29,6,2.84,IR,2022-02-07 04:46:31,Clear
4,4,Isangel,-19.5500,169.2667,78.91,94,100,13.80,VU,2022-02-07 04:49:50,Clouds


In [23]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [24]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
4,4,Isangel,-19.5500,169.2667,78.91,94,100,13.80,VU,2022-02-07 04:49:50,Clouds
7,7,Hithadhoo,-0.6000,73.0833,82.62,67,23,14.25,MV,2022-02-07 04:49:52,Clouds
8,8,Bambous Virieux,-20.3428,57.7575,81.21,78,100,10.87,MU,2022-02-07 04:49:53,Clouds
9,9,Tucano,-10.9631,-38.7867,71.96,93,78,4.47,BR,2022-02-07 04:49:54,Clouds
14,14,Esperance,-33.8667,121.9000,76.84,44,11,23.29,AU,2022-02-07 04:49:56,Clouds
23,23,Airai,-8.9266,125.4092,71.83,74,57,2.55,TL,2022-02-07 04:50:00,Clouds
27,27,Port Alfred,-33.5906,26.8910,76.55,70,56,1.61,ZA,2022-02-07 04:50:02,Clouds
29,29,Biak,-0.9131,122.8766,85.51,62,90,2.44,ID,2022-02-07 04:50:03,Clouds
31,31,Touros,-5.1989,-35.4608,76.10,78,13,8.30,BR,2022-02-07 04:50:04,Clouds
39,39,Tecoanapa,16.5167,-98.7500,76.73,77,23,5.08,MX,2022-02-07 04:50:08,Clouds


In [25]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Date                   0
Current Description    0
dtype: int64

In [26]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_nonull_df = preferred_cities_df.dropna()

In [28]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_nonull_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Isangel,VU,78.91,Clouds,-19.5500,169.2667,
7,Hithadhoo,MV,82.62,Clouds,-0.6000,73.0833,
8,Bambous Virieux,MU,81.21,Clouds,-20.3428,57.7575,
9,Tucano,BR,71.96,Clouds,-10.9631,-38.7867,
14,Esperance,AU,76.84,Clouds,-33.8667,121.9000,
23,Airai,TL,71.83,Clouds,-8.9266,125.4092,
27,Port Alfred,ZA,76.55,Clouds,-33.5906,26.8910,
29,Biak,ID,85.51,Clouds,-0.9131,122.8766,
31,Touros,BR,76.10,Clouds,-5.1989,-35.4608,
39,Tecoanapa,MX,76.73,Clouds,16.5167,-98.7500,


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))